In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/feature_engineered_data.csv', low_memory=False)

In [3]:
df.shape

(40347, 149)

In [4]:
col_to_analyse = [
    "Gen_Demande",
    "Gen_Active",
    "Horizon_Activ",
    "ZCOM_SR_CL_MIMPOTS",
    "Mrev_Tit",
    "Nb_Enf",
    "Ressource",
    "Charge",
    "Ancbanc_Tit",
    "DMA",
    "MDPRBDUREE",
    "Mdecouvert",
    "Ancien_Banc_Tit",
    "Ancien_Prof_Tit",
    "RAV",
    "Ratio_Ress_RAV",
    "RAV_Rev",
    "Tx_Edt",
    "Age_Tit",
    "Flag_CTX",
    "Flag_Actif",
    "Fraudeur",
    "ACP",
    "SS",
    "REF",
    "Flag_Finance",
    "Entite",
    "Decision_Finale",
    "MCLFCHAB1",
    "MCLFCSITFAM",
    "CSP_Tit",
    "MDPRCPRODCOM",
    "MDPRCTYPEPROD",
    "BP"
]

In [5]:
potential_vars = ['ZCOM_SR_CL_MIMPOTS','Mrev_Tit','Mrev_Cj','Nb_Enf','Ressource','Charge','Ancbanc_Tit','Ancbanc_Cj',
                  'DMA','MDPRBDUREE','Mdecouvert','Ancien_Banc_Tit','Ancien_Banc_Cj','Ancien_Prof_Tit',
                  'Ancien_Prof_Cj','Ancien_Habit','RAV','Ratio_Ress_RAV','RAV_Rev','Tx_Edt','Age_Tit','Age_Cj','Flag_CTX',
                  'Flag_Actif','Fraudeur','ACP','SS','REF','Flag_Finance','Decision_Finale','MCLFCHAB1','MCLFCSITFAM',
                  'CSP_Tit','CSP_Cj','MDPRCPRODCOM','MDPRCTYPEPROD','BP', 'Entite', 'Gen_Active', 'Gen_Demande', 'Horizon_Activ']
df.head()

,TYPEPRODUIT,TDPRNFOY,TDPRNCONT,TDPRNCARTE,TDPRNDT,TDPRDCREAT,TDOCNAUTO,TDPRNSOC,TDPRCPRODCOM,TDOCDDECIOCT,...,RAV,Ratio_Ress_RAV,RAV_Rev,Tx_Edt,DMA,Mdecouvert,Defaut_B,Defaut_C,MP,BP
0,REV,35330,15486,16521,30697,20160809,14375,10349,AD7,20160811,...,1700.0,1.000000,NaN,0.000000,800.0,800.0,0,0,0,1
1,REV,18953,28271,32815,13965,20160711,16375,11137,AD7,20160711,...,2059.0,1.000000,NaN,0.000000,800.0,800.0,0,0,0,1
2,REV,40043,31191,8914,40101,20150424,12089,26024,AD7,20150429,...,1555.0,1.128617,0.974922,0.113960,800.0,800.0,0,0,0,1
3,REV,1969,32342,30083,15636,20141016,32387,24698,AD7,20141021,...,2080.0,1.000000,1.106383,0.000000,800.0,800.0,0,0,0,1
4,REV,13764,7971,30292,28885,20140822,30672,31851,AD7,20140828,...,3734.0,1.080343,0.998930,0.074368,800.0,800.0,0,0,0,1


## Preprocessing

**DMA** amount cannot exceed 3000€.

In [6]:
df["DMA"] = np.where(df["DMA"] > 3000, df["DMA"].median(), df["DMA"])

**Applicant’s age** must be between 18 and 85.

In [7]:
df["Age_Tit"] = np.where((df["Age_Tit"]<18) | (df["Age_Tit"]>85), df["Age_Tit"].median(), df["Age_Tit"])

**Professional seniority** must not exceed 55 years.

In [8]:
df["Ancien_Prof_Tit"] = np.where(df["Ancien_Prof_Tit"] > 55, 999999, df["Ancien_Prof_Tit"])

**Taxes amount** must not exceed 10 000€

In [9]:
df["ZCOM_SR_CL_MIMPOTS"] = np.where(df["ZCOM_SR_CL_MIMPOTS"] > 10000, df["ZCOM_SR_CL_MIMPOTS"].median(), df["ZCOM_SR_CL_MIMPOTS"])

**Income amount** must not exceed 100 000€.

In [10]:
df["Mrev_Tit"] = np.where(df["Mrev_Tit"] > 100000, df["Mrev_Tit"].median(), df["Mrev_Tit"])
df["Mrev_Cj"] = np.where(df["Mrev_Cj"] > 100000, df["Mrev_Tit"].median(), df["Mrev_Cj"])

## Quick analysis

In [11]:
def VarAnalysis(col):
    missing_values = df[col].isna().sum()
    total_values = df[col].count() + missing_values
    percentage_missing = (missing_values / total_values) * 100
    print(df[col].value_counts().head())
    # print("The IV is : ", information_value(df[col], df['BP']).sum())
    print("Le pourcentage de valeurs manquantes est de : ",percentage_missing, "%")
    if df[col].dtypes == 'float64' or df[col].dtypes == 'int64':
        print("La valeur maximale est : ", df[col].max())
        print("La valeur minimale est : ", df[col].min())
        print("La valeur moyenne est : ", df[col].mean())

In [12]:
for var in potential_vars :
    VarAnalysis(var)

ZCOM_SR_CL_MIMPOTS
0.0      13061
53.0       740
51.0       641
21.0       613
117.0      577
Name: count, dtype: int64
Le pourcentage de valeurs manquantes est de :  0.0 %
La valeur maximale est :  9404.0
La valeur minimale est :  0.0
La valeur moyenne est :  132.52234366867424
Mrev_Tit
1200.0    2569
1500.0    2090
2000.0    2046
1300.0    1548
1800.0    1469
Name: count, dtype: int64
Le pourcentage de valeurs manquantes est de :  0.1908444246164523 %
La valeur maximale est :  80000.0
La valeur minimale est :  0.0
La valeur moyenne est :  1830.4052892972436
Mrev_Cj
0.0       25895
2000.0      714
1500.0      672
1200.0      584
3000.0      409
Name: count, dtype: int64
Le pourcentage de valeurs manquantes est de :  14.19188539420527 %
La valeur maximale est :  40000.0
La valeur minimale est :  0.0
La valeur moyenne est :  479.80052569249875
Nb_Enf
0.0    2117
1.0    1065
2.0    1003
3.0     336
4.0      74
Name: count, dtype: int64
Le pourcentage de valeurs manquantes est de :  88.54

c:\Users\aysim\Documents\Ynov\EpreuvesCertif\BLOC5\Octroi\score-octroi\.venv\Lib\site-packages\numpy\_core\_methods.py:51: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


On remarque que les variables **Ancbanc_Tit** et **Ancbanc_Cj** ont des statistiques quasiment identiques en termes dee distribution des valeurs, de valeurs maximale, minimale et moyenne. Seul le pourcentage de valeurs manquantes diffèrent : **6.17% pour Ancbanc_Cj** et **4.5% pour Ancbanc_Tit**

On remarque que la variable **MDPRBDUREE** est remplie de valeurs nulles.


## Missing Values

Remplissons les **valeurs manquantes des variables** qui nous intéressent pour la suite.

In [13]:
# Vérification des valeurs manquantes
na_counts = df[potential_vars].isna().sum()
na_pct = (na_counts / len(df) * 100).round(2)
missing_summary = (
    pd.DataFrame({'nb_na': na_counts, 'pct_na': na_pct})
      .sort_values('nb_na', ascending=False)
)

print(f"Nombre de lignes: {len(df)}")
display(missing_summary)
print("\nVariables avec au moins une valeur manquante:")
print(missing_summary[missing_summary.nb_na > 0])

Nombre de lignes: 40347


,nb_na,pct_na
Ancien_Prof_Cj,38947,96.53
Ancien_Habit,36249,89.84
Age_Cj,35861,88.88
Nb_Enf,35727,88.55
CSP_Cj,22921,56.81
Horizon_Activ,7030,17.42
Gen_Active,7030,17.42
RAV_Rev,5795,14.36
Mrev_Cj,5726,14.19
Ancbanc_Cj,2491,6.17



Variables avec au moins une valeur manquante:
                 nb_na  pct_na
Ancien_Prof_Cj   38947   96.53
Ancien_Habit     36249   89.84
Age_Cj           35861   88.88
Nb_Enf           35727   88.55
CSP_Cj           22921   56.81
Horizon_Activ     7030   17.42
Gen_Active        7030   17.42
RAV_Rev           5795   14.36
Mrev_Cj           5726   14.19
Ancbanc_Cj        2491    6.17
Ancien_Banc_Cj    2491    6.17
Ancien_Banc_Tit   1841    4.56
Ancbanc_Tit       1841    4.56
CSP_Tit            672    1.67
MCLFCHAB1          585    1.45
MCLFCSITFAM        553    1.37
Ancien_Prof_Tit    114    0.28
Mrev_Tit            77    0.19
Age_Tit              5    0.01
Tx_Edt               1    0.00
Ratio_Ress_RAV       1    0.00


In [14]:
df["Nb_Enf"] = df["Nb_Enf"].fillna(0)

df["Mrev_Tit"] = df["Mrev_Tit"].fillna(df["Mrev_Tit"].median())

df["Ancbanc_Tit"] = df["Ancbanc_Tit"].fillna(df["Ancbanc_Tit"].median())

df["Ancien_Banc_Tit"] = df["Ancien_Banc_Tit"].fillna(df["Ancien_Banc_Tit"].median())

df["Ancien_Prof_Tit"] = df["Ancien_Prof_Tit"].fillna(df["Ancien_Prof_Tit"].median())

df["Ratio_Ress_RAV"] = df["Ratio_Ress_RAV"].fillna(df["Ratio_Ress_RAV"].median())

df["RAV_Rev"] = df["RAV_Rev"].fillna(df["RAV_Rev"].median())

df["Age_Tit"] = df["Age_Tit"].fillna(df["Age_Tit"].median())

df["Tx_Edt"] = df["Tx_Edt"].fillna(df["Tx_Edt"].median())

df["MCLFCHAB1"] = df["MCLFCHAB1"].fillna('Missing')

df["MCLFCSITFAM"] = df["MCLFCSITFAM"].fillna('Missing')

df["CSP_Tit"] = df["CSP_Tit"].fillna('Missing')

## Save

In [15]:
df.to_csv("../data/preprocessed_data.csv", index=False)